In [ ]:
import numpy as np
import yfinance as yf
import pandas_datareader as pdr
import datetime as dt

# MonteCarlo simulation function for Geometric Brownian Motion or Jump Diffusion
def monte_carlo_simulation(num_simulations, time_steps, mu, sigma, initial_price, diffusion_type='GBM', mu_j=0, sigma_j=0, lambda_=0):
    '''
    This function performs a Monte Carlo simulation of stock prices. It takes in the following parameters:

    num_simulations: the number of simulations to run
    time_steps: the number of time steps to simulate over
    mu: the mean of the log returns
    sigma: the standard deviation of the log returns
    initial_price: the initial stock price
    diffusion_type: the type of diffusion to use, either 'GBM' (Geometric Brownian Motion) or 'Jump' (Jump Diffusion). Default is 'GBM'.
    mu_j: the mean of the jump component. Default is 0.
    sigma_j: the standard deviation of the jump component. Default is 0.
    lambda_: the frequency of jumps. Default is 0.

    The function creates an array to store the simulated prices and sets the initial price for all simulations. 
    It then sets the time step and chooses the type of diffusion. If the diffusion type is 'GBM', 
    it uses the GBM equation to calculate the next price for each simulation. If the diffusion type is 'Jump', 
    it uses the Jump Diffusion equation to calculate the next price for each simulation. The function returns an array of simulated prices.    
    '''    
    # Create an array to store the simulated prices
    sim_prices = np.zeros((num_simulations, time_steps))
    # Set the initial price for all simulations
    sim_prices[:, 0] = initial_price
    # Set the time step
    dt = 1 / time_steps
    # Choose the type of diffusion
    if diffusion_type == 'GBM':
        for i in range(1, time_steps):
            # Generate random numbers for each simulation
            rand = np.random.normal(0, 1, num_simulations)
            # Use GBM to calculate the next price for each simulation
            sim_prices[:, i] = sim_prices[:, i-1] * np.exp((mu - sigma**2 / 2) * dt + sigma * np.sqrt(dt) * rand)
    elif diffusion_type == 'Jump':
        for i in range(1, time_steps):
            # Generate random numbers for each simulation
            rand = np.random.normal(0, 1, num_simulations)
            # Generate random numbers for the jump component
            jump = np.random.normal(mu_j, sigma_j, num_simulations)
            # Use Jump Diffusion to calculate the next price for each simulation
            sim_prices[:, i] = sim_prices[:, i-1] * np.exp((mu - sigma**2 / 2 - lambda_ * (np.exp(mu_j + sigma_j ** 2 / 2) - 1)) * dt + sigma * np.sqrt(dt) * rand + jump)
    else:
        raise ValueError('Invalid diffusion type. Choose either GBM or Jump.')
    return sim_prices

# Create a plotly function to generate a chart
def plotly_chart():
    
    conf = [1,5,10]
    levels = []
    print(f'For {time_steps} time steps in {num_simulations} simulations')
    for i in conf:
        value_at_risk = np.percentile(sim_prices[:,-1], i)
        levels.append(value_at_risk)
        print(f'VAR at {100-i}% conf. level =  {(value_at_risk):.3f} $')


    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    # Create a figure
    fig = make_subplots(rows=1, cols=2, specs=[[{'type':'scatter'}, {'type':'histogram'}]], subplot_titles=[f"Price at start {(initial_price):.2f} [$]", "Distribution"], column_widths=[2,1])
    
    # Add lines for each simulation
    for i in range(num_simulations):
        fig.add_trace(go.Scatter(x=np.arange(time_steps), y=sim_prices[i,:], mode='lines'), row=1, col=1)# name='Simulation {}'.format(i+1)
    
    # Add a line for the average price
    fig.add_trace(go.Scatter(x=np.arange(time_steps), y=np.mean(sim_prices, axis=0), mode='lines'), row=1, col=1) # , name='Average Price'
    
    # Add a histogram of the simulated prices and vetical lines
    fig.add_trace(go.Histogram(x=sim_prices[:,-1], name='Simulated Prices'), row=1, col=2)
    fig.add_vline(x=levels[0], name='VAR 99%',  line_width=1, line_dash="dash", line_color="green", row=1, col=2)
    fig.add_vline(x=levels[1], name='VAR 95%',  line_width=1, line_dash="dash", line_color="blue", row=1, col=2)
    fig.add_vline(x=levels[2], name='VAR 90%',  line_width=1, line_dash="dash", line_color="black", row=1, col=2)


    # Set the layout
    fig.update_layout(title=f'Monte Carlo Simulation of Stock Prices for {ticker} with metrics extracted from {start_date} until today',
                      xaxis_title='Steps',
                      yaxis_title='Price',
                      showlegend=False)

    # Show the figure
    fig.show()

# Retreive stock data from desired start date until last available
def get_stock_data(ticker, start_date):
    """
    This function gets the stock data for a given ticker, start date and end date. 
    It uses the yfinance library to download the stock data, calculates the daily logarithmic returns,
    mean rate of return (mu), volatility (sigma) and initial price.
    If the yfinance library fails to retrieve the stock data, it tries to use pandas_datareader library.
    """
    today = dt.date.today()
    end_date = today.strftime("%Y-%m-%d")
    try:
        # Get the stock data
        stock_data = yf.download(ticker, start=start_date, end=end_date)
    except:
        try:
            stock_data = pdr.get_data_yahoo(ticker, start_date, end_date)
        except:
            return "No data found, please try again later"
    # Calculate the daily logarithmic returns
    returns = np.log(stock_data['Adj Close']/stock_data['Adj Close'].shift(1))
    returns = returns.dropna()
    # Calculate the mean rate of return (mu)
    mu = returns.mean()
    # Calculate the volatility (sigma)
    sigma = returns.std()
    # Get the initial price
    initial_price = stock_data['Adj Close'][-1]
    return mu, sigma, initial_price

In [1]:
# Define Stock Parameters

start_date = '2010-01-01'
ticker = ['MSFT', 'TSLA','AMZN']
# mu, sigma, initial_price = get_stock_data(ticker, start_date)

# Montecarlo Parameters
num_simulations = 1000
time_steps = 100
diffusion_type ='GBM' # or 'Jump' and if jump, add mu_j, sigma_j, lambda_ to the function

In [7]:

for i in ticker:
    mu, sigma, initial_price = get_stock_data(i, start_date)

    sim_prices = monte_carlo_simulation(num_simulations, time_steps, mu, sigma, initial_price, diffusion_type)
    # Run the plotly function
    plotly_chart()